### Connect to your own Dropbox folder through dropbox API
- You can get a Dropbox API key here: https://www.dropbox.com/developers
- Click the permissions tab in the API page and check all read and write access under individual scopes
- Go to settings tab and generate an access token, that would be the API key you need
- Save the API key in a .env file in the same directory as this notebook, with the following format: paperApiKey='YOUR_API_KEY'
- Import the required packages

In [1]:
import dropbox
from dotenv import load_dotenv
import os
from pathlib import Path
from tqdm import tqdm
import pypandoc

load_dotenv()
paperApiKey = os.getenv('paperApiKey')
print("Initializing Dropbox API...")
dbx = dropbox.Dropbox(paperApiKey)
try:
    account_info = dbx.users_get_current_account()
    print(f"Dropbox connection successful. Connected as: {account_info.name.display_name}")
except Exception as e:
    print(f"Error connecting to Dropbox: {e}")

Initializing Dropbox API...
Dropbox connection successful. Connected as: Kyle Deng


### Migrating paper files in personal Dropbox
- You will need to install pandoc on your local machine to convert the markdown files to word files
- You can install pandoc from here: https://pandoc.org/installing.html

In [ ]:
def convert_all_dropbox_paper_docs(dbx):
    """
    Searches the entire Dropbox account for .paper files, exports each one as a Markdown file,
    saves the Markdown file locally in the same relative path as its Dropbox location,
    converts it to a Word (DOCX) document saved in that same location, and then deletes the Markdown file.
    
    Assumes that your current working directory is the base for the local mirror of your Dropbox structure.
    
    Parameters:
        dbx (dropbox.Dropbox): An authenticated Dropbox client.
    """
    # List all files in the Dropbox account recursively.
    result = dbx.files_list_folder("", recursive=True)
    paper_files = [entry for entry in result.entries
                   if isinstance(entry, dropbox.files.FileMetadata) and entry.name.lower().endswith('.paper')]
    pbar = tqdm(desc="Searching Dropbox pages", unit="page")
    pbar.update(1)
    # Handle pagination.
    while result.has_more:
        result = dbx.files_list_folder_continue(result.cursor)
        paper_files.extend([entry for entry in result.entries
                            if isinstance(entry, dropbox.files.FileMetadata) and entry.name.lower().endswith('.paper')])
        pbar.update(1)
    pbar.close()
    
    print(f"Found {len(paper_files)} Paper files in your Dropbox account.")

    # Retrieve the current user's account information.
    user_name = dbx.users_get_current_account().name.display_name

    # Process each .paper file.
    for file_meta in tqdm(paper_files, desc="Converting Paper files", unit="file"):
        # Determine the base file name (without extension).
        base_name = os.path.splitext(file_meta.name)[0]
        
        # Build a local directory that mirrors the Dropbox file's path.
        # file_meta.path_lower is something like "/Folder1/Subfolder/filename.paper"
        # We remove the leading slash and get the directory part.
        relative_dir = os.path.dirname(file_meta.path_lower).lstrip("/")
        local_dir = os.path.join(Path(os.path.join(os.path.expanduser("~"), f"Arca Dropbox/{user_name}/")), relative_dir)
        os.makedirs(local_dir, exist_ok=True)
        
        # Define local file paths.
        local_docx_path = os.path.join(local_dir, base_name + ".docx")
        
        # Export the Paper file as Markdown using Dropbox API.
        try:
            md_metadata, md_res = dbx.files_export(file_meta.path_lower, export_format="markdown")
            # md_res.content is in bytes; decode it to a string.
            md_text = md_res.content
        except Exception as e:
            tqdm.write(f"Error exporting {file_meta.path_lower} as Markdown: {e}")
            continue

        # Convert the Markdown text to DOCX content in memory.
        try:
            docx_text = pypandoc.convert_text(md_text, 'docx', format='md')
        except Exception as e:
            tqdm.write(f"Error converting markdown to DOCX for {file_meta.path_lower}: {e}")
            continue

        # Now write the DOCX content to disk.
        try:
            with open(local_docx_path, "wb") as f:
                f.write(docx_text.encode('utf-8'))
        except Exception as e:
            tqdm.write(f"Error saving DOCX file {local_docx_path}: {e}")

convert_all_dropbox_paper_docs(dbx)

Searching Dropbox pages: 103page [01:09,  1.47page/s]


Found 2 Paper files in your Dropbox account.


Converting Paper files:  50%|█████     | 1/2 [00:01<00:01,  1.19s/file]

Error converting markdown to DOCX for /2024 smart churning minutes.paper: Output to docx only works by using a outputfile.


Converting Paper files: 100%|██████████| 2/2 [00:02<00:00,  1.29s/file]

Error converting markdown to DOCX for /paper docs/task tracking - data analytics_org.paper: Output to docx only works by using a outputfile.
